<a href="https://colab.research.google.com/github/aliabbas70/Plant-Disease-Prediction-and-fertilizer-recommendation/blob/main/Fertilizer_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Optimized Fertilizer Recommendation System
# Using XGBoost with Feature Engineering & Crop-specific Models


import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor


# Step 1: Load Dataset

data = pd.read_csv("/content/data_core.csv")  # Replace with your CSV path
print("Dataset Loaded:", data.shape)

# Step 2: Encode Categorical Columns

le_crop = LabelEncoder()
le_soil = LabelEncoder()

data['Crop Type'] = le_crop.fit_transform(data['Crop Type'])
data['Soil Type'] = le_soil.fit_transform(data['Soil Type'])

crop_names = dict(zip(le_crop.transform(le_crop.classes_), le_crop.classes_))


# Step 3: Feature Engineering

data['Temp_Humidity'] = data['Temparature'] * data['Humidity']
data['Moisture_Soil'] = data['Moisture'] / (data['Soil Type'] + 1)

features = ['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Temp_Humidity', 'Moisture_Soil']
targets = ['Nitrogen', 'Phosphorous', 'Potassium']


# Step 4: Prepare Crop-specific Models and Scalers

final_predictions = []
crop_models = {} # Dictionary to store models for each crop
crop_scalers = {} # Dictionary to store scalers for each crop

for crop_code in data['Crop Type'].unique():
    crop_data = data[data['Crop Type'] == crop_code]
    X = crop_data[features]
    y = crop_data[targets]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    crop_scalers[crop_code] = scaler # Store scaler

    # Step 5: Hyperparameter Tuning

    param_grid = {
        'n_estimators': [200, 300, 400],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.01, 0.03, 0.05],
        'subsample': [0.7, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 1.0],
        'gamma': [0, 0.1, 0.2],
        'reg_alpha': [0, 0.1, 0.5],
        'reg_lambda': [1, 1.5, 2]
    }

    xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
    random_search = RandomizedSearchCV(xgb, param_distributions=param_grid,
                                       n_iter=20, cv=3, scoring='neg_mean_absolute_error', verbose=0)
    random_search.fit(X_train_scaled, y_train)

    best_model = random_search.best_estimator_
    crop_models[crop_code] = best_model # Store model


    # Step 6: Predict & Evaluate

    y_pred = best_model.predict(X_test_scaled)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"Crop: {crop_names[crop_code]}, MAE: {mae:.3f}, RMSE: {rmse:.3f}")


    # Step 7: Map Predictions to Fertilizer

    def map_to_fertilizer(pred):
        N, P, K = pred
        fertilizers = []
        if N > 0: fertilizers.append(f"Urea: {round(N,1)} kg/ha")
        if P > 0: fertilizers.append(f"DAP: {round(P,1)} kg/ha")
        if K > 0: fertilizers.append(f"MOP: {round(K,1)} kg/ha")
        return ', '.join(fertilizers)

    recommendations = [map_to_fertilizer(p) for p in y_pred]

    crop_results = X_test.copy()
    crop_results['Predicted NPK'] = list(y_pred)
    crop_results['Fertilizer Recommendation'] = recommendations

    final_predictions.append(crop_results)


# Step 8: Combine Results

final_df = pd.concat(final_predictions)
final_df.reset_index(drop=True, inplace=True)

# Example: show first 10 recommendations
print("\nSample Fertilizer Recommendations:")
print(final_df[['Fertilizer Recommendation']].head(10))

Dataset Loaded: (8000, 9)
Crop: Maize, MAE: 8.180, RMSE: 10.346
Crop: Sugarcane, MAE: 8.379, RMSE: 10.671
Crop: Cotton, MAE: 8.705, RMSE: 11.217
Crop: Tobacco, MAE: 8.561, RMSE: 10.862
Crop: Paddy, MAE: 8.657, RMSE: 11.112
Crop: Barley, MAE: 8.048, RMSE: 10.279
Crop: Wheat, MAE: 8.221, RMSE: 10.667
Crop: Millets, MAE: 7.877, RMSE: 10.197
Crop: Oil seeds, MAE: 8.271, RMSE: 10.803
Crop: Pulses, MAE: 8.682, RMSE: 10.901
Crop: Ground Nuts, MAE: 8.089, RMSE: 10.338

Sample Fertilizer Recommendations:
                           Fertilizer Recommendation
0  Urea: 22.0 kg/ha, DAP: 12.199999809265137 kg/h...
1  Urea: 23.600000381469727 kg/ha, DAP: 16.399999...
2  Urea: 19.399999618530273 kg/ha, DAP: 19.0 kg/h...
3  Urea: 15.0 kg/ha, DAP: 21.899999618530273 kg/h...
4  Urea: 16.5 kg/ha, DAP: 19.5 kg/ha, MOP: 3.0999...
5  Urea: 15.699999809265137 kg/ha, DAP: 18.200000...
6  Urea: 15.199999809265137 kg/ha, DAP: 14.300000...
7  Urea: 16.5 kg/ha, DAP: 17.100000381469727 kg/h...
8  Urea: 15.1999998092

In [ ]:
user_input = {
    "Temperature": 26,
    "Humidity": 52,
    "Moisture": 38,
    "Soil Type": "Sandy",   # categorical
    "Crop Type": "Maize"    # categorical
}


In [ ]:
user_input['Soil Type'] = le_soil.transform([user_input['Soil Type']])[0]
user_input['Crop Type'] = le_crop.transform([user_input['Crop Type']])[0]


In [ ]:
user_input['Temp_Humidity'] = user_input['Temperature'] * user_input['Humidity']
user_input['Moisture_Soil'] = user_input['Moisture'] / (user_input['Soil Type'] + 1)


In [ ]:
import pandas as pd

input_df = pd.DataFrame([{
    'Temperature': user_input['Temperature'],
    'Humidity': user_input['Humidity'],
    'Moisture': user_input['Moisture'],
    'Soil Type': user_input['Soil Type'],
    'Crop Type': user_input['Crop Type'],
    'Temp_Humidity': user_input['Temp_Humidity'],
    'Moisture_Soil': user_input['Moisture_Soil']
}])


In [ ]:
input_scaled = scaler.transform(input_df)


In [ ]:
# Retrieve the correct model and scaler for the user's specified crop type
crop_code = user_input['Crop Type']
model = crop_models[crop_code]
scaler = crop_scalers[crop_code]

# Scale the input data using the correct scaler
input_scaled = scaler.transform(input_df)

# Make a prediction using the correct model
prediction = model.predict(input_scaled)

# Map the prediction to a fertilizer recommendation
def map_to_fertilizer(pred):
    N, P, K = pred
    fertilizers = []
    if N > 0: fertilizers.append(f"Urea: {round(N,1)} kg/ha")
    if P > 0: fertilizers.append(f"DAP: {round(P,1)} kg/ha")
    if K > 0: fertilizers.append(f"MOP: {round(K,1)} kg/ha")
    return ', '.join(fertilizers)

recommendation = map_to_fertilizer(prediction[0])

print("\nFertilizer Recommendation for your input:")
print(recommendation)


Fertilizer Recommendation for your input:
Urea: 23.600000381469727 kg/ha, DAP: 13.399999618530273 kg/ha, MOP: 3.200000047683716 kg/ha
